<a href="https://colab.research.google.com/github/PedromfRibeiro/GGCD/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install all
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark
from IPython.display import clear_output
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
clear_output()

In [17]:
#!/usr/bin/python3.7
import findspark
from pyspark import SparkContext
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
findspark.init()
if 'sc' in  locals():
    sc.stop()
import sys
from time import time
from pyspark.sql.functions import split, explode
from pyspark import SparkConf
from pyspark.sql import *
import pyspark.sql.functions as f
from pyspark.sql import functions as F

from pyspark.sql.functions import lit,udf, struct,col, when ,rank
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark import StorageLevel
from pyspark.streaming import StreamingContext
from IPython.core.display import display, HTML
#import org.apache.spark.sql.types.DateType
#spark.conf.set("spark.sql.session.timeZone", "Portugal/Lisbon")
import pandas as pd
import urllib
import gzip
conf = (SparkConf()
 .setMaster("local[*]")
 .setAppName("data_frame")
 .set("spark.driver.cores","15")
 .set("spark.executor.instances", "15")
 .set("spark.executor.cores", "15")
 .set("spark.dynamicAllocation.enabled", "false")
 .set("spark.shuffle.service.enabled", "false")
 .set("spark.driver.maxResultSize", "20G")
 .set("spark.driver.memory","4G")
 .set("spark.executor.memory", "4G"))
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)
sqlContext.clearCache()
sc._conf.getAll()
spark = SparkSession.builder.appName('EXAMPLE').getOrCreate()


In [3]:
#Donwload File
from os import path
def Donwload_FilesFULL(name):
    if path.exists("/content/"+name+".gz") is False:
        url = "https://datasets.imdbws.com/"+name+".gz"
        !wget -q $url 
    if path.exists("/content/"+name) is False:
        unzip = name+".gz"
        !gzip -d $unzip
def Donwload_FilesMINI():
    if path.exists("/content/title.basics.tsv.bz2") is  False:
        !wget -q url https://storage.googleapis.com/ggcdimdb/mini/title.basics.tsv.bz2
    if path.exists("/content/title.basics.tsv") is False:
        !bzip2 -d title.basics.tsv.bz2

#Exercicio 2.a) Top Generos

In [ ]:
t0=time()
#Donwload do ficheiro
Donwload_FilesFULL("title.basics.tsv")
#Leitura do ficheiro
df = spark.read.csv("/content/title.basics.tsv", sep=r'\t', header=True)
#Remover dados nâo essenciais e seperar a coluna generos
df1 = (df["startYear","genres"]).withColumn('genres',explode(split('genres',',')))
#alterar o startYear para apenas contabilizar as decadas e não os anos
df1 = df1.withColumn('startYear', df['startYear'].substr(1, 3))
#Remover dados em branco
df1 = df1.filter((df.startYear != '\\N')&(df.genres != '\\N'))
df1 = df1.groupby('startYear','genres').count()
output = df1.groupBy(F.col('startYear')).agg(F.first('genres').alias('genero'),F.max('count').alias('Quantidade')).orderBy(['startYear'])
output.cache()
#spark.catalog.clearCache()
output.show()
print("Total:\n"+"-"*40+"\n"+(str(time() - t0))+"\n"+"-"*40)

+---------+-----------+----------+
|startYear|     genero|Quantidade|
+---------+-----------+----------+
|      187|Documentary|         4|
|      188|      Sport|        55|
|      189|      Sport|      6041|
|      190|      Music|     25476|
|      191|    Mystery|     58857|
|      192|     Horror|     14021|
|      193|    History|      9304|
|      194|    Western|      7735|
|      195|      Adult|     41147|
|      196|Documentary|     62467|
|      197|Documentary|    101598|
|      198|      Adult|    120630|
|      199|      Short|    195214|
|      200|      Short|    404800|
|      201|  Talk-Show|    777901|
|      202|      Drama|     87785|
+---------+-----------+----------+

None
Total:
----------------------------------------
25.018404722213745
----------------------------------------


#Exercicio 3.a) Base

In [ ]:
t0=time()

#Donwload Dos ficheiros
Donwload_FilesFULL("name.basics.tsv")
Donwload_FilesFULL("title.principals.tsv")
Donwload_FilesFULL("title.basics.tsv")
Donwload_FilesFULL("title.ratings.tsv")
#Leitura dos ficheiro
df_actor = spark.read.csv("/content/name.basics.tsv",      sep=r'\t', header=True)
df_year  = spark.read.csv("/content/title.basics.tsv",     sep=r'\t', header=True)
df_title = spark.read.csv("/content/title.principals.tsv", sep=r'\t', header=True)
df_rting = spark.read.csv("/content/title.ratings.tsv",    sep=r'\t', header=True)
#Remover dados nâo essenciais
df_title = df_title['tconst','nconst']
df_year  = df_year ['tconst','primaryTitle','startYear']
df_actor = df_actor['nconst','primaryName','birthYear','deathYear']
df_actor = df_actor.replace('\\N',"2021")

#Join dos ficheiros
joined1 = df_title.join(df_year,on='tconst')
joined1 = joined1.join(df_rting,on='tconst')
joined1 = joined1.join(df_actor,on='nconst')
#Resultado pretendido
joined2 = joined1.groupBy('nconst').agg(
    F.first('primaryName').alias('Nome'),
    (F.max('deathYear')-F.max('birthYear')).alias('Idade'),
    F.count('tconst').alias('Nº Presenças'),
    (F.max('startYear')-F.min('startYear')).alias('Na Atividade'),
    F.mean('averageRating').cast(DecimalType(3)).alias('Media de rating')
    )
joined2.cache()
#spark.catalog.clearCache()
joined2.show()
print("Total:\n"+"-"*40+"\n"+(str(time() - t0))+"\n"+"-"*40)
#de notar que a função Cast tem uma performace muito negativa para o benchmark do processo, sem ela a media ir a dar com varias casas decimais.

+---------+--------------------+-----+------------+------------+---------------+
|   nconst|                Nome|Idade|Nº Presenças|Na Atividade|Media de rating|
+---------+--------------------+-----+------------+------------+---------------+
|nm0000086|      Louis de Funès| 69.0|          59|        70.0|              7|
|nm0000198|         Gary Oldman| 63.0|         120|        37.0|              7|
|nm0000354|          Matt Damon| 51.0|         232|        41.0|              6|
|nm0000362|        Danny DeVito| 77.0|         293|        48.0|              7|
|nm0000493|         Jack Lemmon| 76.0|         146|        66.0|              7|
|nm0000767| Jean-Hugues Anglade| 66.0|         120|        37.0|              7|
|nm0000847|          Theda Bara| 70.0|          27|        98.0|              6|
|nm0001079|    Scatman Crothers| 76.0|         142|        61.0|              7|
|nm0001105|       Linda Darnell| 42.0|          54|        60.0|              7|
|nm0002222| Christopher Brad

# Exercicio 3.b) Hits

In [19]:
t0=time()

#Donwload Dos ficheiros
Donwload_FilesFULL("name.basics.tsv")
Donwload_FilesFULL("title.basics.tsv")
Donwload_FilesFULL("title.ratings.tsv")
Donwload_FilesFULL("title.principals.tsv")

#Leitura dos ficheiro
df_actor = spark.read.csv("/content/name.basics.tsv",      sep=r'\t', header=True)
df_movie  = spark.read.csv("/content/title.basics.tsv",     sep=r'\t', header=True)
df_title = spark.read.csv("/content/title.principals.tsv", sep=r'\t', header=True)
df_rting = spark.read.csv("/content/title.ratings.tsv",    sep=r'\t', header=True)
#Remover dados nâo essenciais
df_movie  = df_movie ['tconst','primaryTitle']
df_actor = df_actor['nconst','primaryName']
df_title = df_title['tconst','nconst']
df_rting = df_rting['tconst','averageRating']

joined1 = df_title.join(df_movie,on='tconst')
joined1 = joined1.join(df_rting,on='tconst')
joined1 = joined1.join(df_actor,on='nconst')


window = Window.partitionBy(joined1['nconst']).orderBy(joined1['averageRating'].desc())

result = joined1.select('*', rank().over(window).alias('rank')) .filter(col('rank') <= 10)
result.cache()
#spark.catalog.clearCache()
result.show()

print("Total:\n"+"-"*40+"\n"+(str(time() - t0))+"\n"+"-"*40)

+---------+----------+--------------------+-------------+--------------+----+
|   nconst|    tconst|        primaryTitle|averageRating|   primaryName|rank|
+---------+----------+--------------------+-------------+--------------+----+
|nm0000086| tt3533882|Louis de Funès, d...|          8.6|Louis de Funès|   1|
|nm0000086| tt8768374|Le gendarme et l'...|          8.3|Louis de Funès|   2|
|nm0000086| tt0060474|La Grande Vadrouille|          7.9|Louis de Funès|   3|
|nm0000086| tt2949354|   Monsieur de Funès|          7.7|Louis de Funès|   4|
|nm0000086|tt12600988|La folle aventure...|          7.6|Louis de Funès|   5|
|nm0000086| tt0062083|               Oscar|          7.5|Louis de Funès|   6|
|nm0000086| tt0804908|Louis de Funès ou...|          7.5|Louis de Funès|   6|
|nm0000086| tt4307300|Louis de Funès et...|          7.5|Louis de Funès|   6|
|nm0000086| tt0069747|The Mad Adventure...|          7.4|Louis de Funès|   9|
|nm0000086| tt0049877|      Four Bags Full|          7.4|Louis d